In [108]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pandas as pd

In [109]:
# only run this once ever per session
# try:
#     whole = pd.read_csv("signs.csv")

# except:
#     print("No dataset found")

In [110]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
cap = cv2.VideoCapture(0)
ls = []
elapsed = 0

with mp_hands.Hands(
    model_complexity = 0,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as hands:
    
    while cap.isOpened():

        time_start = time.time()

        retval, frame = cap.read()

        if not retval:
            print("Camera Error; Exiting")
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        if results.multi_hand_world_landmarks:
            for hand_landmarks in results.multi_hand_world_landmarks:
                ls.append([[lmk.x, lmk.y, lmk.z] for lmk in hand_landmarks.landmark])


        fps = 1 / (time.time() - time_start)

        frame = cv2.putText(frame, f"FPS: {round(fps)}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow("Hand Landmarks", frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1736377202.532832  196395 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736377202.533756  205787 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.3.6), renderer: Mesa Intel(R) HD Graphics 630 (KBL GT2)
W0000 00:00:1736377202.549615  205781 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736377202.555589  205779 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0x1e04070) is not the object's thread (0x2a98110).
Cannot move to target thread (0x1e04070)

QObject::moveToThread: Current thread (0x1e04070) is not the object's thread (0x2a98110).
Cannot move to target thread (0x1e04070)

QObject::moveToThread: Current thread (0x1e04070) is not the object's thread (0x2a98110).
Cannot move to target

In [112]:
# normalize coordinates to be relative centered at landmark 0
ls = np.array(ls)

for l in ls:
    for i in range(21):
        l[i][0] = l[i][0] - l[0][0]
        l[i][1] = l[i][1] - l[0][1]

ls

array([], dtype=float64)

In [113]:
reshaped_ls = ls.copy()
reshaped_ls = reshaped_ls.reshape(-1, 42)
reshaped_ls.shape

(0, 42)

In [114]:
df_columns = []
for i in range(21):
    df_columns.append(f"Landmark {i} x")
    df_columns.append(f"Landmark {i} y")

In [115]:
# create dataframe where rows are frames and columns are coordinates of all 21 landmarks (x, y have distinct columns)
df = pd.DataFrame(reshaped_ls, columns=df_columns)
df.shape

(0, 42)

In [116]:
# add label for handpose (DO NOT USE EMOJIS LOL)
df["Label"] = "okay"

In [117]:
try:
    whole = pd.concat([whole, df], axis=0)
except:
    whole = df

whole.shape

(486, 43)

In [123]:
whole["Label"].value_counts()

Label
good    486
Name: count, dtype: int64

In [119]:
# just in case of bad data
def delete_label(df, label):
    return df[df["Label"] != label]

# whole = delete_label(whole, "okay")

In [125]:
whole.to_csv("new_signs_en.csv", index=False)